In [181]:
import numpy as np

In [182]:
class Regularizations:

    class L2:
        @staticmethod
        def reg(a):
            return np.mean(a**2)

        @staticmethod
        def reg_prime(a):
            return 2 * a / a.size

In [234]:
class Losses:

    class MSE:
        @staticmethod
        def loss(y_true, y_pred):
            return np.mean(np.power(y_true-y_pred, 2))

        @staticmethod
        def loss_prime(y_true, y_pred):
            # print(y_true.shape, y_pred.shape)
            return 2*(y_pred-y_true) / y_true.size

In [235]:
class Activations: # alphabet order

    class activation:
        @staticmethod
        def activation(a):
            return a

        @staticmethod
        def activation_prime(a):
            return 1


    class ReLU(activation):
        @staticmethod
        def activation(a):
            return np.maximum(a, 0)

        @staticmethod
        def activation_prime(a):
            ret = np.array(1 * (a > 0))
            return ret


    class sigmoid(activation):
        @staticmethod
        def activation(a):
            return 1 / (1 + np.exp(-a))

        @staticmethod
        def activation_prime(a):
            return a * (1 - a)


    class softmax(activation):
        @staticmethod
        def activation(a):
            exp = np.exp(a)
            return exp / (0.0001 + np.sum(exp, axis=0)) # mb 0

        @staticmethod
        def activation_prime(a):
            t = np.eye(N=a.shape[0], M=a.shape[1])
            return t * a * (1 - a) - (1 - t) * a * a


    class stable_softmax(activation):
        @staticmethod
        def activation(a):
            a = a - max(a)
            exp = np.exp(a)
            return exp / np.sum(exp, axis=1)

        # dont know prime


    class tanh(activation):
        @staticmethod
        def activation(a):
            return np.tanh(a)

        @staticmethod
        def activation_prime(a):
            return 1 - np.tanh(a)**2

In [236]:
class Layers:

    class DummyLayer:

        def __init__(self):
            self.input_shape = None
            self.output_shape = None

        def forward_pass(self, input):
            raise NotImplementedError

        def backward_pass(self, output):
            raise NotImplementedError



    class Dense(DummyLayer):

        def __init__(self, input_shape=None, output_shape=None, learning_rate=None, reg_const=None, reg_type=None):
            super().__init__()
            self.input_shape = input_shape
            self.output_shape = output_shape

            self.input = None
            self.output = None

            self.learning_rate = learning_rate
            self.reg_const = reg_const

            self.reg_type = reg_type

            if self.reg_type is None:
                self.reg_function = None
                self.reg_prime = None
            else:
                self.reg_function = reg_type.reg
                self.reg_prime = reg_type.reg_prime

            self.features_weights = np.random.rand(input_shape, output_shape) - 0.5
            self.bias_weights = np.random.rand(1, output_shape) - 0.5

            self.learnable = True

        def forward_pass(self, input):
            self.input = input
            self.output = input @ self.features_weights + self.bias_weights
            return self.output

        def backward_pass(self, output_error):
            input_error = output_error @ self.features_weights.T
            weights_error = self.input.T @ output_error + self.reg_const * self.reg_prime(self.features_weights)
            bias_error = np.sum(output_error, axis=0)

            self.features_weights -= self.learning_rate * weights_error
            self.bias_weights -= self.learning_rate * bias_error

            return input_error



    class Activation(DummyLayer):

        def __init__(self, activation_type=Activations.tanh):
            super().__init__()
            self.activation = activation_type.activation
            self.activation_prime = activation_type.activation_prime

            self.input = None
            self.output = None

            self.learnable = False

        def forward_pass(self, input):
            self.input = input
            self.output = self.activation(self.input)
            return self.output

        def backward_pass(self, output):
            return self.activation_prime(self.input) * output



    class Conv2d(DummyLayer):


In [237]:
class NeuralNetwork:

    def __init__(self, layers, default_learning_rate=0.01, default_reg_const=0.01, reg_type=Regularizations.L2, loss_class=Losses.MSE):
        self.layers = []
        for layer in layers:
            if layer.learnable:
                if layer.learning_rate is None:
                    layer.learning_rate = default_learning_rate

                if layer.reg_const is None:
                    layer.reg_const = default_reg_const

                if layer.reg_type is None:
                    layer.reg_function = reg_type.reg
                    layer.reg_prime = reg_type.reg_prime
            self.layers.append(layer)

        self.loss = loss_class.loss
        self.loss_prime = loss_class.loss_prime

    def fit(self, X, y, cnt_epochs=10, cnt_it=10000): # add optimizer
        it_for_epoch = cnt_it // cnt_epochs
        for i in range(cnt_epochs):
            for j in range(it_for_epoch):

                output = X
                for layer in self.layers:
                    output = layer.forward_pass(output)

                error = self.loss_prime(y, output)

                for layer in reversed(self.layers):
                    error = layer.backward_pass(error)

            print('epoch %d/%d   error=%f' % (i+1, cnt_epochs, self.loss(y, self.predict(X))))

    def predict(self, X):
        output = X
        for layer in self.layers:
            output = layer.forward_pass(output)

        return output

In [239]:
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])

y = np.array([[0],
              [1],
              [1],
              [0]])

print(X.shape, y.shape)

nn = NeuralNetwork([
    Layers.Dense(2, 2),
    Layers.Activation(activation_type=Activations.tanh),
    Layers.Dense(2, 1),
    Layers.Activation(activation_type=Activations.tanh)
], default_learning_rate=0.01, default_reg_const=0)

nn.fit(X, y, cnt_epochs=10, cnt_it=30000)

nn.predict(X)

(4, 2) (4, 1)
epoch 1/10   error=0.192022
epoch 2/10   error=0.093582
epoch 3/10   error=0.007651
epoch 4/10   error=0.002455
epoch 5/10   error=0.001340
epoch 6/10   error=0.000895
epoch 7/10   error=0.000663
epoch 8/10   error=0.000523
epoch 9/10   error=0.000429
epoch 10/10   error=0.000363


array([[0.00120928],
       [0.97305038],
       [0.97318436],
       [0.0020329 ]])